In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
sys.path.append(os.path.abspath('../src/'))
sys.path.append(os.path.abspath('../'))

In [2]:
from src.central_ids import IDS
from src.utils import cal_metric, print_results
from src.dataset import CANDataset
from torch.utils.data import DataLoader
import pytorch_lightning as pl
import numpy as np
from pathlib import Path
from sklearn.metrics import f1_score, confusion_matrix
import pickle
import torch
from flwr.common.parameter import parameters_to_ndarrays
from collections import OrderedDict
from utils import set_parameters, test_model

from pytorch_lightning import loggers
from pytorch_lightning.callbacks import ModelCheckpoint

In [18]:
car_model = 'BMW'
rnd = 44
args = {
    'ckpt_file' : f'../save/FedAvg/train1000_50rnds_3/ckpt-rnd{rnd}.npz',
    'train_dir': f'../../Data/LISA/{car_model}/train1000/',
    'val_dir': f'../../Data/LISA/{car_model}/val/',
    # 'data_dir': f'../../Data/LISA/{car_model}/',
    'save_dir': f'../save/FedAvg_FN/{car_model}/',
    'C': 2,
    'batch_size': 64,
    'epochs': 10,
    'val_freq': 1,
    'lr': 0.005,
    'weight_decay': 1e-4
}

# Fine tuning the federated model

In [19]:
# ckpt_file = f'../save/FedAvg/train1000_100rnds/ckpt-rnd{rnd}.npz'
tensors = np.load(args['ckpt_file'], allow_pickle=True)['arr_0'][()]
data_dir = f'../../Data/LISA/{car_model}/1/'
model = IDS(**args)

In [20]:
weights = parameters_to_ndarrays(tensors)
params_dict = zip(model.state_dict().keys(), weights)
state_dict = OrderedDict({k: torch.tensor(v) for k, v in params_dict})
model.load_state_dict(state_dict, strict=True)

<All keys matched successfully>

In [21]:
for name, param in model.named_parameters():
    if 'fc' in name:
        param.requires_grad = False
    else:
        param.requires_grad = True

In [22]:
logger = loggers.TensorBoardLogger(save_dir=args['save_dir'],) 
checkpoint_callback = ModelCheckpoint(
        filename='{epoch:02d}-{val_loss:.2f}-{val_f1:.4f}',
        every_n_epochs=args['val_freq'],
        monitor='val_loss', 
        save_top_k=5)

In [23]:
trainer = pl.Trainer(max_epochs=args['epochs'], accelerator='cuda', 
                        gpus=[0], logger=logger, log_every_n_steps=100, 
                        check_val_every_n_epoch=args['val_freq'],
                        callbacks=[checkpoint_callback])

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [24]:
trainer.fit(model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]

  | Name      | Type             | Params
-----------------------------------------------
0 | model     | Classifier       | 700 K 
1 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
699 K     Trainable params
258       Non-trainable params
700 K     Total params
2.801     Total estimated model params size (MB)


Set the optiimzer with the lr =  0.005


Sanity Checking: 0it [00:00, ?it/s]

Validation f1 score: 0.6016


Training: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation f1 score: 0.5816


Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


Validation f1 score: 0.7244


In [25]:
data_dir = f'../../Data/LISA/{car_model}/train1000'
test_model(data_dir, model)

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/dhkim1/miniconda3/envs/torch/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py:1767: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=4)`.
  category=PossibleUserWarning,
/home/dhkim1/miniconda3/envs/torch/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:241: PossibleUserWarning: The dataloader, predict_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


Predicting: 0it [00:00, ?it/s]

(0.9330811066445968, 0.07075)

## Run model from checkpoint

In [29]:
ckpt_files = '../save/FedAvg_FN/Kia/lightning_logs/version_4/checkpoints/epoch=04-val_loss=0.02-val_f1=0.9933.ckpt'
fedavg_model = IDS.load_from_checkpoint(ckpt_files, **args)
data_dir = f'../../Data/LISA/Kia/'
print('Training set: ')
print(test_model(Path(data_dir) / 'train1000', fedavg_model))
print('Test set: ')
print(test_model(Path(data_dir) / 'test', fedavg_model))

Training set: 


GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/dhkim1/miniconda3/envs/torch/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py:1767: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=4)`.
  category=PossibleUserWarning,
/home/dhkim1/miniconda3/envs/torch/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:241: PossibleUserWarning: The dataloader, predict_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


Predicting: 0it [00:00, ?it/s]

(1.0, 0.0)
Test set: 


GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/dhkim1/miniconda3/envs/torch/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py:1767: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=4)`.
  category=PossibleUserWarning,
/home/dhkim1/miniconda3/envs/torch/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:241: PossibleUserWarning: The dataloader, predict_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


Predicting: 0it [00:00, ?it/s]

(0.9889303814350494, 0.007045840407470289)


In [30]:
ckpt_files = '../save/FedAvg_FN/Tesla/lightning_logs/version_5/checkpoints/epoch=03-val_loss=0.06-val_f1=0.9795.ckpt'
fedavg_model = IDS.load_from_checkpoint(ckpt_files, **args)
data_dir = f'../../Data/LISA/Tesla/'
print('Training set: ')
print(test_model(Path(data_dir) / 'train1000', fedavg_model))
print('Test set: ')
print(test_model(Path(data_dir) / 'test', fedavg_model))

Training set: 


GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/dhkim1/miniconda3/envs/torch/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py:1767: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=4)`.
  category=PossibleUserWarning,
/home/dhkim1/miniconda3/envs/torch/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:241: PossibleUserWarning: The dataloader, predict_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


Predicting: 0it [00:00, ?it/s]

(0.99347061778001, 0.0065)
Test set: 


GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/dhkim1/miniconda3/envs/torch/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py:1767: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=4)`.
  category=PossibleUserWarning,
/home/dhkim1/miniconda3/envs/torch/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:241: PossibleUserWarning: The dataloader, predict_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


Predicting: 0it [00:00, ?it/s]

(0.9543563532628075, 0.019302302201668845)


In [31]:
ckpt_files = '../save/FedAvg_FN/BMW/lightning_logs/version_7/checkpoints/epoch=04-val_loss=0.57-val_f1=0.7215.ckpt'
fedavg_model = IDS.load_from_checkpoint(ckpt_files, **args)
data_dir = f'../../Data/LISA/BMW/'
print('Training set: ')
print(test_model(Path(data_dir) / 'train1000', fedavg_model))
print('Test set: ')
print(test_model(Path(data_dir) / 'test', fedavg_model))

Training set: 


GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/dhkim1/miniconda3/envs/torch/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py:1767: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=4)`.
  category=PossibleUserWarning,
/home/dhkim1/miniconda3/envs/torch/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:241: PossibleUserWarning: The dataloader, predict_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


Predicting: 0it [00:00, ?it/s]

(0.8330142522867474, 0.19625)
Test set: 


GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/dhkim1/miniconda3/envs/torch/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py:1767: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=4)`.
  category=PossibleUserWarning,
/home/dhkim1/miniconda3/envs/torch/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:241: PossibleUserWarning: The dataloader, predict_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


Predicting: 0it [00:00, ?it/s]

(0.8167505793974267, 0.2820158042001045)
